In [2]:
!pip install arxiv


     ---------------------------------------- 81.3/81.3 KB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for sgmllib3k, since package 'wheel' is not installed.
  Running setup.py install for sgmllib3k: started
  Running setup.py install for sgmllib3k: finished with status 'done'


You should consider upgrading via the 'C:\Users\athar\Projects\Chatbot_openAI_Llama\Chatbot-OpenAI-Llama\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [13]:
from langchain_community.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

In [5]:
!pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
  Running setup.py install for wikipedia: started
  Running setup.py install for wikipedia: finished with status 'done'


You should consider upgrading via the 'C:\Users\athar\Projects\Chatbot_openAI_Llama\Chatbot-OpenAI-Llama\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
# Import Wikipedia API wrapper to fetch data from Wikipedia
api_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)

# This sets up a Wikipedia search tool that will return the top 1 result 
# and limit the content to 200 characters
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
# Now, if you use 'tool.run("your query")', it will fetch a short summary 
# of the most relevant Wikipedia page.
wiki.run("What is machine learning")

'Page: Machine learning\nSummary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)
vectordb=FAISS.from_documents(documents= documents,embedding= OpenAIEmbeddings())

retriever= vectordb.as_retriever()


In [8]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000247FAF60F40>, search_kwargs={})

In [ ]:
# We use retrieval tool because, the agent will have freedom to choose or 
# not to choose the retriever. If needed it will be referred but if not needed, 
# other tools will be used.

from langchain.tools.retriever import create_retriever_tool
retriever_tool= create_retriever_tool(retriever,"langsmith_search",
                      "Search for the information about langsmith. For any questions about Langsmith, you must use this tool")

In [10]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper= arxiv_wrapper)

In [11]:
arxiv.name

'arxiv'

In [16]:
tools=[wiki,arxiv,retriever_tool]

In [17]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\athar\\Projects\\Chatbot_openAI_Llama\\Chatbot-OpenAI-Llama\\venv\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for the information about langsmith. For any questions about Langsmith, you must use this tool', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000247B6B34B80>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIE

In [32]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["OpenAI_API_KEY"]=os.getenv("OpenAI_API_KEY")

llm=ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [80]:
# from langchain import hub
# prompt=hub.pull("hwchase17/openai-functions-agent:51bac8c5")

# prompt.messages
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Define the prompt template with the correct input variables
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    ("human", "{input}"),
    ("system", "{chat_history}"),  # Ensure chat_history is included
    ("system", "{agent_scratchpad}")  # Ensure agent_scratchpad is included
])

input_data = {
    "input": "Tell me about LangSmith",
    "chat_history": "",  # Empty chat history if this is the first query
    "agent_scratchpad": ""  # Initial scratchpad value
}

In [81]:
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [82]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['chat_history'], input_types={}, partial_variables={}, template='{chat_history}'), additional_kwargs={}), SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad'], input_types={}, partial_variables={}, template='{agent_scratchpad}'), additiona

In [85]:
agent_executor.invoke(input_data)



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'LangSmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith is framework-agnostic — it can be used with or without LangChain's open source frameworks langchain and langgraph.If you are using either of these, you can enable LangSmith tracing with a single environment variable.
For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prot

{'input': 'Tell me about LangSmith',
 'chat_history': '',
 'agent_scratchpad': '',
 'output': 'LangSmith is a platform for building production-grade Large Language Model (LLM) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith provides observability features that help you analyze traces, configure metrics, dashboards, and alerts based on these traces. It also offers evaluation capabilities to score application performance and gather human feedback on your data. Additionally, LangSmith supports prompt engineering, which involves iterating on prompts, version control, and collaboration features to find the perfect prompt for your LLM application.\n\nIf you are interested in learning more about LangSmith, you can join the "Interrupt: The Agent AI Conference" by LangChain on May 13 & 14 in San Francisco.'}